**Install Dependencies**

In [1]:
!pip install groq
!pip install requests
!pip install openai requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 4.8 MB/s eta 0:00:00


**Import Libraries**

In [2]:
import os
import openai
import requests

**Add API Key**

In [ ]:
# Replace YOUR_API_KEY_HERE with your actual Groq API key
os.environ["GROQ_API_KEY"] = "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"

# Initialize client
client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

**Checking Available Models**

In [ ]:
api_key = "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"
url = "https://api.groq.com/openai/v1/models"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    models_list = data.get("data", [])
    print("Available models:")
    for model in models_list:
        print(f"- {model['id']}")
else:
    print(f"Failed to fetch models: {response.status_code}")
    print(response.text)

Available models:
- meta-llama/llama-guard-4-12b
- whisper-large-v3-turbo
- meta-llama/llama-prompt-guard-2-86m
- llama-3.1-8b-instant
- groq/compound-mini
- playai-tts-arabic
- meta-llama/llama-4-maverick-17b-128e-instruct
- moonshotai/kimi-k2-instruct
- openai/gpt-oss-120b
- openai/gpt-oss-20b
- whisper-large-v3
- groq/compound
- llama-3.3-70b-versatile
- meta-llama/llama-prompt-guard-2-22m
- qwen/qwen3-32b
- playai-tts
- allam-2-7b
- meta-llama/llama-4-scout-17b-16e-instruct
- moonshotai/kimi-k2-instruct-0905
- gemma2-9b-it
- deepseek-r1-distill-llama-70b


**Initialize conversation history**

In [5]:
# Conversation history storage
conversation_history = []

# Add messages
def add_message(role, content):
    """
    Add a message to the conversation history.

    Args:
        role (str): 'user' or 'assistant'
        content (str): Text content of the message
    """
    conversation_history.append({"role": role, "content": content})


**Truncate conversation history**

In [14]:
def truncate_history(history, max_turns=None, max_chars=None, max_words=None):
    """
    Truncate conversation history based on turns, characters, or words.
    Supports partial truncation for long messages.

    Args:
        history (list): List of message dictionaries
        max_turns (int, optional): Maximum number of conversation turns
        max_chars (int, optional): Maximum total characters
        max_words (int, optional): Maximum total words

    Returns:
        list: Truncated conversation history
    """
    # Truncate by number of turns
    if max_turns:
        history = history[-max_turns:]

    # Truncate by number of characters (partial messages allowed)
    if max_chars:
        total_chars = 0
        truncated = []
        for msg in reversed(history):
            msg_len = len(msg['content'])
            remaining = max_chars - total_chars
            if remaining <= 0:
                break
            if msg_len > remaining:
                # Keep only the last 'remaining' characters of this message
                msg_copy = msg.copy()
                msg_copy['content'] = msg_copy['content'][-remaining:]
                truncated.insert(0, msg_copy)
                break
            truncated.insert(0, msg)
            total_chars += msg_len
        history = truncated

    # Truncate by number of words (partial messages allowed)
    if max_words:
        total_words = 0
        truncated = []
        for msg in reversed(history):
            words = msg['content'].split()
            remaining = max_words - total_words
            if remaining <= 0:
                break
            if len(words) > remaining:
                # Keep only the last 'remaining' words of this message
                msg_copy = msg.copy()
                msg_copy['content'] = ' '.join(words[-remaining:])
                truncated.insert(0, msg_copy)
                break
            truncated.insert(0, msg)
            total_words += len(words)
        history = truncated

    return history


**Summarize conversation using Groq API**

In [15]:
def summarize_history(history):
    messages_text = "\n".join([f"{m['role']}: {m['content']}" for m in history])
    response = client.chat.completions.create(
        model="groq/compound-mini",
        messages=[{"role":"user", "content": f"Summarize this conversation:\n{messages_text}"}],
        temperature=0.3,          # Low randomness for concise summary
        max_completion_tokens=500, # Enough for summary
        top_p=1,
        stream=False
    )
    summary = response.choices[0].message.content
    return summary

**Periodic summarization after every k messagest**

In [18]:
def maybe_summarize(history, k=3):
    """
    Append a summary after every k messages instead of replacing the history.
    """
    if len(history) % k == 0:
        summary = summarize_history(history)
        print(f"\n[Conversation summarized after {k} messages]:\n{summary}\n")
        # Append the summary as a system message
        history.append({"role": "system", "content": f"Summary so far: {summary}"})
    return history


**Demonstration with multiple conversation samples**

In [19]:
sample_conversations = [
    [("user", "Hi, I want to learn about AI."),
     ("assistant", "Sure! What aspect of AI?"),
     ("user", "Neural networks."),
     ("assistant", "Neural networks are models inspired by the human brain."),
     ("user", "Thanks!")],

    [("user", "Hello, I need help with Python."),
     ("assistant", "Python is great! What do you want to know?"),
     ("user", "How to read files."),
     ("assistant", "You can use open() function to read files."),
     ("user", "Got it!")],

    [("user", "Hey, tell me about machine learning."),
     ("assistant", "ML is about training models to learn patterns."),
     ("user", "Give me an example."),
     ("assistant", "Image recognition is a common ML example."),
     ("user", "Thanks!")]
]

# Run demo
for idx, conv in enumerate(sample_conversations):
    print(f"\n--- Conversation Sample {idx+1} ---")

    # Reset conversation history for each sample
    conversation_history = []

    for role, text in conv:
        add_message(role, text)

    # Summarize at the end of this conversation sample (instead of after each message)
    conversation_history = maybe_summarize(conversation_history, k=len(conv))

    # Show different truncation settings
    print("\nTruncated to last 4 messages:")
    truncated_turns = truncate_history(conversation_history, max_turns=4)
    for msg in truncated_turns:
        print(f"{msg['role']}: {msg['content']}")

    print("\nTruncated to last 50 characters:")
    truncated_chars = truncate_history(conversation_history, max_chars=50)
    for msg in truncated_chars:
        print(f"{msg['role']}: {msg['content']}")

    print("\nTruncated to last 20 words:")
    truncated_words = truncate_history(conversation_history, max_words=20)
    for msg in truncated_words:
        print(f"{msg['role']}: {msg['content']}")



--- Conversation Sample 1 ---

[Conversation summarized after 5 messages]:
We've just started. You've expressed interest in learning about AI, specifically neural networks. I've provided a brief introduction. We can continue exploring the topic if you'd like.


Truncated to last 4 messages:
user: Neural networks.
assistant: Neural networks are models inspired by the human brain.
user: Thanks!
system: Summary so far: We've just started. You've expressed interest in learning about AI, specifically neural networks. I've provided a brief introduction. We can continue exploring the topic if you'd like.

Truncated to last 50 characters:
system: We can continue exploring the topic if you'd like.

Truncated to last 20 words:
system: learning about AI, specifically neural networks. I've provided a brief introduction. We can continue exploring the topic if you'd like.

--- Conversation Sample 2 ---

[Conversation summarized after 5 messages]:
The conversation is over. The user asked for help wi